# Register model and deploy locally

See: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-local/register-model-deploy-local.ipynb

- Register model
- Deploy the image as a web service in a local Docker container.
- Quickly test changes to your entry script by reloading the local service.
- Optionally, you can also make changes to model, conda or extra_docker_file_steps and update local service

## Assumptions

- Data is already preprocessed in `data/`

# Setup Workspace and Prereqs

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.50.0


In [3]:
from azureml.core.workspace import Workspace

# the config file is one directory up
# NOTE: this is an interactive authorization!
ws = Workspace.from_config('..')
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


joshua_nanostics_ml
jnji-rg
westus
2676283c-665b-4d2f-bf73-2c380edf47d9


# Train Model

the code is ripped from `../train.ipynb`, just without the MLFlow stuff

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd 

def split_and_scale(df):
    y = df["diagnosis_01"]
    X = df.drop(["diagnosis_01"],axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, X.columns

In [5]:
from sklearn.svm import SVC

def train_svc(df):
    X_train, X_test, y_train, y_test, train_cols = split_and_scale(df)
    svc = SVC()
    svc.fit(X_train, y_train)
    return svc, X_test, y_test, train_cols

In [6]:
import pandas as pd 


data = '../data/cleaned-wisconsin-lof.parquet'
df = pd.read_parquet(data)

svc, X_test, y_test, train_cols = train_svc(df)

In [7]:
# df_test = pd.DataFrame(X_test, columns=df.columns.drop('diagnosis_01'))

# d = df_test.iloc[0].to_dict()
# # # pd.DataFrame(d, index=[0])
# # svc.predict(pd.DataFrame(d, index=[0]))
X_test[0]

array([1.06440132, 0.80405382, 1.4890773 , 1.82441732, 2.12011885,
       1.62621261, 1.67428659, 2.99057306, 1.00350134, 3.10485027,
       2.83317058, 0.33334666, 1.20575539, 1.43235646, 1.31676276,
       0.79819672, 0.75820626, 0.60151745, 1.59568557, 1.56051245,
       0.06110285, 0.51173261, 1.04502958, 1.17239422, 0.59875684,
       0.57992707])

In [8]:
import joblib

joblib.dump(svc, 'model.pkl')

['model.pkl']

## Register Model

In [9]:
from azureml.core.model import Model

model = Model.register(model_path="model.pkl",
                       model_name="wisconsin-BCa-model",
                       tags={'area': "cancer", 'type': "svc"},
                       description="SVC model to predict Breast Cancer",
                       workspace=ws)

Registering model wisconsin-BCa-model


## Create Environment + Inference Config

straight copy-paste

In [10]:
import sklearn

from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [13]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=environment)

## Deploy Model as a Docker Service

This is the good stuff!!!!

In [14]:
from azureml.core.webservice import LocalWebservice

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

/tmp/ipykernel_3224/1718659047.py:6: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)
ModelNotFound: Model with id wisconsin-BCa-model:4 not found in provided workspace



Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 45fea3aaa51047319e877b6733c4b3ed.azurecr.io
Logging into Docker registry 45fea3aaa51047319e877b6733c4b3ed.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 45fea3aaa51047319e877b6733c4b3ed.azurecr.io/azureml/azureml_4c8b2de40556312ba6c5a5b1465c00ab
 ---> 7d8e4100ac92
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 79f3f9ad06bd
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjI2NzYyODNjLTY2NWItNGQyZi1iZjczLTJjMzgwZWRmNDdkOSIsInJlc291cmNlR3JvdXBOYW1lIjoiam5qaS1yZyIsImFjY291bnROYW1lIjoiam9zaHVhX25hbm9zdGljc19tbCIsIndvcmtzcGFjZUlkIjoiNDVmZWEzYWEtYTUxMC00NzMxLTllODctN2I2NzMzYzRiM2VkIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> [Warning] The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was r

In [15]:
print('Local service port: {}'.format(local_service.port))


Local service port: 6789


## Test Web Service


In [13]:
print(local_service.get_logs())


/bin/bash: /azureml-envs/azureml_c804bc43f202681eba163665491706a1/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c804bc43f202681eba163665491706a1/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c804bc43f202681eba163665491706a1/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-05-16T16:51:22,270989801+00:00 - rsyslog/run 
2023-05-16T16:51:22,272249384+00:00 - nginx/run 
bash: /azureml-envs/azureml_c804bc43f202681eba163665491706a1/lib/libtinfo.so.6: no version information available (required by bash)
2023-05-16T16:51:22,314987259+00:00 - gunicorn/run 
2023-05-16T16:51:22,347727884+00:00 | gunicorn/run | 
2023-05-16T16:51:22,382501384+00:00 | gunicorn/run | ###############################################
2023-05-16T16:51:22,413155634+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-05-16T16:51:22,441842509+00:00 | gunico

In [24]:
import json

sample_input = json.dumps({
    'data': X_test[0:10].tolist()
})

# local_service.run({ 'data': X_test[0:10].tolist() })
local_service.run(sample_input)

ConnectionError: HTTPConnectionPool(host='localhost', port=6789): Max retries exceeded with url: /score (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xffff613a9dc0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [29]:
 X_test[0:2]

array([[ 1.06440132,  0.80405382,  1.4890773 ,  1.82441732,  2.12011885,
         1.62621261,  1.67428659,  2.99057306,  1.00350134,  3.10485027,
         2.83317058,  0.33334666,  1.20575539,  1.43235646,  1.31676276,
         0.79819672,  0.75820626,  0.60151745,  1.59568557,  1.56051245,
         0.06110285,  0.51173261,  1.04502958,  1.17239422,  0.59875684,
         0.57992707],
       [-0.5717181 , -2.31970186, -1.47899644, -1.05175858, -1.15248909,
        -1.15452931, -1.25625737, -1.1784716 , -0.91527307, -1.19667305,
        -0.85104217, -1.23083649, -1.21640805, -0.8682195 , -1.4359442 ,
        -0.62149239, -0.87326335, -0.82629925, -0.40398055, -0.36784753,
        -2.1137895 , -1.29354244, -1.13316074, -1.28738793, -0.75765468,
        -1.24889433]])

In [26]:
import requests

input_data = json.dumps({
    'data': X_test[0:10].tolist()
})
headers = {'Content-Type': 'application/json'}

scoring_uri = "http://localhost:6789/score"
resp = requests.post(scoring_uri, input_data, headers=headers)
print("prediction:", resp.text)

ConnectionError: HTTPConnectionPool(host='localhost', port=6789): Max retries exceeded with url: /score (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xffff61267910>: Failed to establish a new connection: [Errno 111] Connection refused'))

## Delete Service

In [27]:
local_service.delete()

Container has been successfully cleaned up.
